# Query the dataset

## Environment setting

In this section, we prepare the environment to access the connectomic dataset using the CAVEclient. The CAVEclient is a Python API for querying segment and synapse data hosted on the CAVE infrastructure.

In [90]:
from caveclient import CAVEclient
url = "https://global.brain-wire-test.org/"
datastack = "fish1_full"
client = CAVEclient(datastack_name=datastack, server_address=url)

In [91]:
# check current materialized version
client.materialize.version

327

In [92]:
# get all available tables
client.materialize.get_tables()

['synapses_axax', 'synapses_axde', 'soma_ids_locations']

## Find the current segment ID for a cell body id

Each neuron is identified by a cell body ID, but due to proofreading and edits, the associated segment ID can change over time. To get the most up-to-date segment ID, please input the cell body ID to the `cell_id`.

In [93]:
# query soma by lores_id
cell_id = "123702"
cell_df = client.materialize.query_table(table="soma_ids_locations",
                                filter_equal_dict = {'tag': cell_id})
cell_df

,id,created,superceded_id,valid,tag,pt_supervoxel_id,pt_root_id,pt_position
0,10002,2025-04-04 20:10:25.214941+00:00,NaN,t,123702,78041069207421047,864691128615524977,"[43648, 16000, 3171]"


In [94]:
cell_df['pt_root_id'].values[0]

864691128615524977

The pt_root_id is the current segment ID for the cell. It should be used in downstream queries, such as retrieving synaptic partners.

## Find the pre/post-synaptic partners for a segment ID

Fish1 has two synapse tables. `synapses_axax` is the synaptic connections between axon and axon. `synapses_axde` is the synaptic connections between axon and dendrite.

To find the presynaptic partner for the segment ID, please use 'pre_pt_root_id' for the filter_equal_dict in the query.

In [95]:
root_id = cell_df['pt_root_id'].values[0]
post_partners = client.materialize.query_table(table="synapses_axax",
                                    filter_equal_dict= {'pre_pt_root_id':root_id}) # cell is a pre synaptic partner
print(f"{len(post_partners)} post synaptic partners found when cell {cell_id} with root ID {root_id} was pre synaptic partner")
post_partners.head()

8 post synaptic partners found when cell 123702 with root ID 864691128615524977 was pre synaptic partner


,id,created,superceded_id,valid,size,pre_pt_supervoxel_id,pre_pt_root_id,post_pt_supervoxel_id,post_pt_root_id,pre_pt_position,post_pt_position,ctr_pt_position
0,5113500,2024-12-06 22:57:03.066101+00:00,NaN,t,NaN,78533856776422415,864691128615524977,78533856776421761,864691128617782255,"[23728, 8930, 3528]","[23723, 8909, 3528]","[nan, nan, nan]"
1,6208175,2024-12-06 22:58:48.862219+00:00,NaN,t,NaN,78604294240076081,864691128615524977,78604294240075929,864691128607265958,"[23864, 9082, 3469]","[23852, 9103, 3474]","[nan, nan, nan]"
2,1251625,2024-12-06 22:51:12.951214+00:00,NaN,t,NaN,78604294240076081,864691128615524977,78604294240075957,864691128618084391,"[23871, 9093, 3497]","[23863, 9079, 3503]","[nan, nan, nan]"
3,7290765,2024-12-06 23:00:52.251318+00:00,NaN,t,NaN,78533788325380225,864691128615524977,78533788325380457,864691128607813456,"[23645, 8554, 3975]","[23645, 8539, 3981]","[nan, nan, nan]"
4,1812986,2024-12-06 22:52:11.013828+00:00,NaN,t,NaN,78534062800635873,864691128615524977,78463694056458467,864691128621158405,"[23564, 9664, 3312]","[23550, 9663, 3314]","[nan, nan, nan]"


To find the postsynaptic partner for the segment ID, please use 'post_pt_root_id' for the filter_equal_dict in the query.

In [96]:
root_id = cell_df['pt_root_id'].values[0]
pre_partners = client.materialize.query_table(table="synapses_axax",
                                    filter_equal_dict= {'post_pt_root_id':root_id}) # cell is a post synaptic partner
print(f"{len(pre_partners)} pre synaptic partners found when cell {cell_id} with root ID {root_id} was post synaptic partner")
pre_partners.head()

48 pre synaptic partners found when cell 123702 with root ID 864691128615524977 was post synaptic partner


,id,created,superceded_id,valid,size,pre_pt_supervoxel_id,pre_pt_root_id,post_pt_supervoxel_id,post_pt_root_id,pre_pt_position,post_pt_position,ctr_pt_position
0,9494203,2024-12-06 23:04:13.606565+00:00,NaN,t,NaN,78533925495898393,864691128618084391,78533925495898290,864691128615524977,"[23693, 9031, 3535]","[23681, 9013, 3532]","[nan, nan, nan]"
1,6204097,2024-12-06 22:58:48.862219+00:00,NaN,t,NaN,78533788325380457,864691128607813456,78533788325380225,864691128615524977,"[23645, 8539, 3981]","[23645, 8554, 3975]","[nan, nan, nan]"
2,6580270,2024-12-06 22:59:31.819412+00:00,NaN,t,NaN,78604088350081059,864691128632354668,78604088350081047,864691128615524977,"[23882, 8229, 3986]","[23879, 8196, 3987]","[nan, nan, nan]"
3,6948690,2024-12-06 23:00:16.061789+00:00,NaN,t,NaN,78533925495898393,864691128618084391,78533925495898290,864691128615524977,"[23788, 9128, 3484]","[23770, 9144, 3472]","[nan, nan, nan]"
4,2527149,2024-12-06 22:53:15.469946+00:00,NaN,t,NaN,78533788258272026,864691128599411286,78533788258272313,864691128615524977,"[23601, 8657, 3907]","[23579, 8647, 3900]","[nan, nan, nan]"


## Find which partners have a soma

The soma table allows us to verify which partner segments belong to such neurons.

In [97]:
# Get all post partner root IDs
post_partners_ids = post_partners['post_pt_root_id'].unique()
post_partners_ids

# Query soma table for these root IDs
post_partners_soma_df = client.materialize.query_table(
    table="soma_ids_locations",
    filter_in_dict={'pt_root_id': post_partners_ids}
)

print(f"{len(post_partners_soma_df)} out of {len(post_partners_ids)} partners have a soma")
post_partners_soma_df

0 out of 8 partners have a soma


,id,created,superceded_id,valid,tag,pt_supervoxel_id,pt_root_id,pt_position


In [98]:
# Get all post partner root IDs
pre_partners_ids = pre_partners['pre_pt_root_id'].unique()
pre_partners_ids

# Query soma table for these root IDs
pre_partners_soma_df = client.materialize.query_table(
    table="soma_ids_locations",
    filter_in_dict={'pt_root_id': pre_partners_ids}
)

print(f"{len(pre_partners_soma_df)} out of {len(pre_partners_ids)} partners have a soma")
pre_partners_soma_df

0 out of 34 partners have a soma


,id,created,superceded_id,valid,tag,pt_supervoxel_id,pt_root_id,pt_position


This filters the list of partners to include only those with a soma annotation.